In [1]:
import os, sys
sys.path.append('..')
from src.utils.dataset import (QM9Dataset, FPolyDatasetV1, FPolyDatasetV2, 
                               basic_collate_fn, collate_fn,
                               to_tensor, to_numpy)
from src.model.modules import SingleEncoderModel, DualEncoderModel, IMaxEncoderModel
from torch_geometric.nn import global_add_pool

import numpy as np
import torch

#qds = QM9Dataset(norm=True)
#qds.generate('./dataset/qm9.csv')
#fds1 = FPolyDatasetV1(norm=True)
#fds1.generate('./dataset/fpolymers_221123.train.csv')
fds2 = FPolyDatasetV2(norm=True)
fds2.generate('./dataset/fpolymers_221123.train.csv')

  Overwriting attribute : norm
  Overwriting attribute : data
  Overwriting attribute : atom_feat_name
  Overwriting attribute : bond_feat_name
  Overwriting attribute : mol_feat_name
  Overwriting attribute : target_desc
  Overwriting attribute : tag
  Overwriting attribute : cache_fn
  Setting new attribute : unique_data


In [2]:
encoder_params = {
    'graph_net_params':{
        'node_dim':26,
        'edge_dim':5,
        'output_dim':24
    }, 
    'mol_net_params':{
        'input_dim':399,
        'output_dim':24
    }
}
decoder_params = {
    'hidden_dim':64,
    'n_layer':4,
    'output_dim':1
}

In [3]:
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

#qdl = DataLoader(qds, batch_size=128, collate_fn=collate_fn)
#f1dl = DataLoader(fds1, batch_size=32, collate_fn=collate_fn)
f2dl = DataLoader(fds2, batch_size=32, collate_fn=collate_fn)

In [13]:
from src.utils.data import CrossValidation
n_fold = 5
cv = CrossValidation(n_fold=n_fold, data=fds2, return_index=True)
for i in range(n_fold):
    t, v = cv[i]
    print(len(t), len(v))

195 49
195 49
195 49
195 49
196 48


In [90]:
tgt = collate_fn(fds2)[1][[5, 2, 1, 6, 10, 52]].view(-1)
tgt, tgt.argsort()

(tensor([50.8500, 76.5200, 86.9500, 47.4600, 70.6600, 86.6600]),
 tensor([3, 0, 4, 1, 5, 2]))

In [5]:
#for feats_f1, targets_f1, ids_f1 in f1dl:
#    continue
for feats_f2, targets_f2, ids_f2 in f2dl:
    continue
#for feats_q, targets_q, ids_q in qdl:
#    continue

In [7]:
from torch.utils.tensorboard import SummaryWriter
os.makedirs('./test/model/graph', exist_ok=True)
writer = SummaryWriter('./test/model/graph')


#model = SingleEncoderModel('concat', encoder_params, decoder_params)
model = DualEncoderModel('concat', encoder_params, decoder_params)
#model = IMaxEncoderModel('concat', encoder_params)
#h1 = model.encoder_A(**feats_f2[0]) 
#h2 = model.encoder_B(**feats_f2[1])
#o1 = global_add_pool(h1 * feats_f2[0]['weight'], feats_f2[0]['data_idx'])
#o2 = global_add_pool(h2 * feats_f2[1]['weight'], feats_f2[1]['data_idx'])
#p1 = model.decoder(o1 + o2)
#p2 = model(feats_f2)
#diff = p1 - p2
with torch.no_grad():
    writer.add_graph(model, input_to_model=[feats_f2])

In [76]:
dl = DataLoader(torch.arange(100), batch_size=10, 
                sampler=SubsetRandomSampler(np.arange(40)))
for batch in dl:
    print(batch)

tensor([37, 38, 36,  9, 31,  1, 25, 16,  4,  6])
tensor([ 0, 23,  5, 35,  2, 22, 18, 10, 39,  7])
tensor([26, 20, 17, 24, 19, 15, 32, 14, 27,  3])
tensor([13, 28, 21, 30, 29, 34, 11,  8, 33, 12])


tensor([[72.1600]])